KAKURO2

[algospot의 KAKURO2 링크](https://algospot.com/judge/problem/read/KAKURO2)  
  
### 문제  
![](http://algospot.com/media/judge-attachments/c546f52aba0339eb3808dbfc11de334a/500px-Kakuro_black_box.svg_032__040_1_041_.png)  
카쿠로는 흔히 십자말풀이 수학 버전이라고 불리는 논리 퍼즐이다. 카쿠로는 위와 같이 생긴 정사각형의 게임판을 가지고 하는 퍼즐로, 이 게임판의 각 칸은 흰 칸이거나, 검은 칸이거나, 힌트 칸이다. (힌트 칸은, 대각선으로 갈라져 있고 숫자가 씌여 있는 칸을 말한다) 모든 흰 칸에 적절히 숫자를 채워 넣어 규칙을 만족시키는 것이 이 게임의 목표이다.  
  
카쿠로의 규칙은 다음과 같다.  
  
- 모든 흰 칸에는 1 부터 9 까지의 정수를 써넣어야 한다.  
- 세로로 연속한 흰 칸들의 수를 모두 더하면, 그 칸들의 바로 위에 있는 힌트 칸의 왼쪽 아래에 씌인 숫자가 나와야 한다.  
- 가로로 연속한 흰 칸들의 수를 모두 더하면, 그 칸들의 바로 왼쪽에 있는 힌트 칸의 오른쪽 위에 씌인 숫자가 나와야 한다.  
- 이 때 한 줄을 이루는 연속된 흰 칸들에는 같은 수를 두 번 넣을 수 없다. 예를 들어, 두 개의 흰 칸의 숫자를 더해서 4가 나와야 한다고 하면, 1+3=4 이거나 3+1=4 만이 가능하고, 2+2=4 는 불가능하다.  
  
### 출력   
N줄에, 각 N개의 정수로 카쿠로 퍼즐의 해답을 출력합니다. 힌트 칸이나 검은 칸은 0 으로 주어지고, 흰 칸은 해당 칸에 들어갈 숫자로 주어진다.  

solution의  
N : 게임판의 크기 (1 <= N <= 20)  
color : 게임판의 색 정보를 가지는 행렬 (0은 해당칸이 검은칸 or 힌트. 1은 흰 칸)  
Q : 힌트의 수  
hints : (y좌표, x좌표, 방향, 합)이 주어진 리스트 (direction 0 : 가로 힌트, 1 : 세로 힌트)  

In [281]:
from collections import Counter
from itertools import combinations
from copy import deepcopy

def generateCandidates(candidates): # 넣을수 있는 후보의 수 구하는 함수
    for i in range(0, 1024, 2): # 힌트에 포함되는 칸에 들어간 수들을 뜻하는 비트마스크
        bit = bin(i)[2:].zfill(10)
        line_sum = 0
        length = 0
        for j in range(9):
            if bit[j] == '1':
                length += 1 
                line_sum += 9-j
        subset = i
        while True:
            candidates[length][line_sum][subset] |= (i & ~subset)
            #  힌트의 길이, 힌트 줄의 합, 현재 힌트에 포함되는 칸에 들어간 수에 대해서
            #  후보 비트마스크를 넣는다.
            if subset == 0:
                break
            subset = (subset - 1) & i # 요소 1개씩 바꿔가며 저장
        
def put(y, x, val, hint_num, value, hint_info): # 해당 좌표에 val값을 넣는다.
    for h in range(2):
        hint_info[hint_num[y][x][h]][2] += 1 << val
    value[y][x] = val
    
def remove(y, x, val, hint_num, value, hint_info): # 해당 좌표에 val값을 빼고, 흰 칸으로 비운다.
    for h in range(2):
        hint_info[hint_num[y][x][h]][2] -= 1 << val
    value[y][x] = 0
    
def getCandHint(hint, candidates, hint_info):
    return candidates[hint_info[hint][1]][hint_info[hint][0]][hint_info[hint][2]]
    # 해당 힌트 길이, 힌트의 합, 그리고 현재 쓰여져있는 수들에 대한 비트마스크 정보에 대해서 힌트에 넣을 수 있는 수들의 비트마스크 반환

def getCandCoord(y, x, hint_num, candidates, hint_info):
    return getCandHint(hint_num[y][x][0], candidates, hint_info) & getCandHint(hint_num[y][x][1], candidates, hint_info)
    # 세로 힌트와 가로 힌트에 대해서 넣을 수 있는 숫자들에 대한 비트마스크를 반환해줌.
    
def search(N, color, value, hint_num, hint_info, candidates):

    y = -1
    x = -1
    minCands = 1023
    
    for i in range(N):
        for j in range(N):
            if color[i][j] == 1 and value[i][j] == 0: # 현재 빈칸이고, 값이 0일 때
                cands = getCandCoord(i, j, hint_num, candidates, hint_info)
                # 현재 (i,j)칸에 넣을 수 있는 숫자들의 비트마스크
                if(Counter(bin(minCands)[2:])['1'] > Counter(bin(cands)[2:])['1']):
                # 만약 넣을 수 있는 수가 더 적을 때
                    minCands = cands # 최소 후보자를 갱신
                    y = i
                    x = j
                    # 그리고 최소 후보자를 지닌 좌표 갱신
    if minCands == 0:
        return False
    if y == -1: # 집어넣을 값이 없다면 (모두 채워졌다면)
        return True

    for val in range(1,10):
        if minCands & (1 << val): # 만약 현재 넣을 수 있는 수 중에 val이 있을 때
            put(y, x, val, hint_num, value, hint_info) # 집어 넣고   
            if search(N, color, value, hint_num, hint_info, candidates): # 탐색
                return True
            remove(y, x, val, hint_num, value, hint_info)
            # 집어 넣은 값이 정답이 아니라면 해당 val을 제거
    return False

def solution(N, color, Q, hints):
    candidates = [[[0 for _ in range(1024)] for _ in range(46)] for _ in range(10)]
    # 한 힌트당 들어갈 수 있는 수는 1~9까지이며, 1~9까지 중복되지 않은 최대 합은 45이며,
    # 비트마스크로 표현한 1~9까지 들어갈 수 있는 최대의 수는 이진수로 111111111 = 511입니다.
    value = [[0 for _ in range(N)] for _ in range(N)]
    # 카쿠로 보드의 값 (정답)
    hint_num = [[[0 for _ in range(2)] for _ in range(N)] for _ in range(N)]
    # 힌트의 정보에 따라 빈칸에 들어갈 수 있는 숫자 정보
    hint_info = [[] for _ in range(Q)] # 힌트의 정보
    generateCandidates(candidates)
    # 힌트에 들어갈 수 있는 길이, 힌트의 합, 현재 힌트내에 들어간 숫자들의 비트마스크
    # 정보에 따라 현재 들어간 숫자들을 비트마스크로 저장함

    
    for i in range(len(hints)):
        hint_length = 0 # 힌트의 길이
        if hints[i][2] == 0: # 가로힌트
            for j in range(hints[i][1], N):
                if color[hints[i][0]-1][j] == 1: # 빈칸일 때
                    hint_length += 1 # 놓을 수 있는 length 1 증가
                else:
                    break
            for j in range(hints[i][1], hints[i][1]+hint_length):
                hint_num[hints[i][0]-1][j][0] = i
                # hint_num에는 (y좌표, x좌표, 방향)에 따라 놓을 수 있는 숫자를 저장
                hint_info[i] = [hints[i][3], hint_length, 0]
                # hint_info에  [힌트의 합계, 힌트의 길이, 현재 이미 쓰인 수에 대한 비트마스크]
            
        else: # 1 -> 세로힌트
            for j in range(hints[i][0], N):
                if color[j][hints[i][1]-1] == 1: # 빈칸일 때
                    hint_length += 1 # 놓을 수 있는 length 1 증가
                else:
                    break
            for j in range(hints[i][0], hints[i][0]+hint_length):
                hint_num[j][hints[i][1]-1][1] = i
                # hint_num에는 (y좌표, x좌표, 방향)에 따라 놓을 수 있는 숫자를 저장
                hint_info[i] = [hints[i][3], hint_length, 0]
                # hint_info에  [힌트의 합계, 힌트의 길이, 현재 이미 쓰인 수에 대한 비트마스크]
        
    if search(N, color, value, hint_num, hint_info, candidates): # 만약 찾을 수 있었다면
        return value # 답을 반환
                

In [282]:
N = 8
color = [[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 0, 0, 1, 1, 1],
         [0, 1, 1, 0, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 1, 0, 0],
         [0, 0, 1, 1, 0, 1, 1, 0],
         [0, 0, 0, 1, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 0, 1, 1],
         [0, 1, 1, 1, 0, 0, 1, 1]]
Q = 24
hints = [[2, 1, 0, 16],
[2, 5, 0, 24],
[3, 1, 0, 17],
[3, 4, 0, 29],
[4, 1, 0, 35],
[5, 2, 0, 7],
[5, 5, 0, 8],
[6, 3, 0, 16],
[7, 1, 0, 21],
[7, 6, 0, 5],
[8, 1, 0, 6],
[8, 6, 0, 3],
[1, 2, 1, 23],
[1, 3, 1, 30],
[1, 6, 1, 27],
[1, 7, 1, 12],
[1, 8, 1, 16],
[2, 5, 1, 17],
[3, 4, 1, 15],
[4, 7, 1, 12],
[5, 5, 1, 7],
[5, 8, 1, 7],
[6, 2, 1, 11],
[6, 3, 1, 10]]

In [283]:
solution(N, color, Q, hints)

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 7, 0, 0, 8, 7, 9],
 [0, 8, 9, 0, 8, 9, 5, 7],
 [0, 6, 8, 5, 9, 7, 0, 0],
 [0, 0, 6, 1, 0, 2, 6, 0],
 [0, 0, 0, 4, 6, 1, 3, 2],
 [0, 8, 9, 3, 1, 0, 1, 4],
 [0, 3, 1, 2, 0, 0, 2, 1]]